
---
<big><big><big><big><big><big>Sieci neuronowe 2018</big></big></big></big></big></big>

---





---
<big><big><big><big><big>Sieci konwolucyjne</big></big></big></big></big>

---



---

<id=tocheading><big><big><big><big>Spis treści</big></big></big></big>
<div id="toc"></div>

---

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

In [ ]:
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

plt.style.use("fivethirtyeight")

from bokeh.io import gridplot, output_file, show
from bokeh.plotting import figure, output_notebook
from bkcharts import Scatter

In [ ]:
output_notebook()

In [ ]:
sns.set(font_scale=2.0)

Image inclusion
<img src="nn_figures/" width="100%">

# Sieci konwolucyjne
1. wykorzystują __konwolucje__ zamiast mnożenia macierzy w co najmniej jednej z warstw
  * czym są _konwolucje_?
  * czym jest _pooling_?

## Noecognitron (Fukushima, 1980)
<img src="nn_figures/neocognitron.png" width="100%"> ['O'Reilly]
1. na wejściu jest __retina__ odpowiedzialna za odbiór obrazu
2. hierarchicznie ułożnone warstwy __S__ i __C__
  * __S__-warstwy (S jak simple) odpowiedzialne za ekstrakcję cech
    * zmienne wejścia modyfikowane poprzez uczenie
    * po zakończeniu uczenia każda s-komórka staje się _ekstraktorem_ pewnej konkretnej cechy w polu widzenia (selektywnie na nią odpowiada)
      * uczenie odpowiada wyborowi wykrywanej cechy
    * cechy bardziej lokalne są wykrywane bliżej wejścia modelu, te bardziej globalne później
  * __C__-warstwy (C jak complex) 
    * pozwalają na korekcję błędów translacji na wejściu
    * wejścia do C-komórek z ekstraktorów w warstwach S są ustalone i niezmienne
    * każda C-komórka dostaje wejście z grupy S-komórek wykrywających __tą samą__ cechę ale niewiele różniących się pozycjach - zapewnia inwariantność na translacje
    * C-komórka jest aktywowana jeśli co najmniej jedna S-komórka wykryła cechę
3. warstwy S i C przypominają komórki w układzie widzenia

<img src="nn_figures/neocognitron2.gif" width="80%"> [ScholarPedia, Fukushima]
1. Sieć uczy się w procesie __samo-organizacji__
  * tylko S-komórki mają modyfikowane wagi
  * __winner-take-all__ wśród komórek w małym okreslonym obszarze (kolumnie) tylko jedna staje się zwycięzcą i jest aktywowana
    * połączenia zwycięzcy są wzmacniane
    * siła wzmocnienia jest proporcjonalna do aktywacji
    * na początku wszystkie połączenia są bardzo słabe
    * po jakimś czasie S-komórki uczą się rozpoznawać pewne wzorce
  * wszystkie komórki z otoczenia zwycięzcy podążają za nim
<img src="nn_figures/neocognitron4.gif" width="100%"> [ScholarPedia, Fukushima]
2. Neocognitron osiąga na zbiorze MNIST błąd rzędu 2.5%
3. Nauczanie jest niezależne dla każdej warstwy
  * istotne stają się wielkości komórek C i S, co utrudnia uczenie

## LeNet5 (LeCun 1998)
<img src="nn_figures/lenet5.pdf" width="100%">[LeCun NIPS]

## Konwolucje
1. dla dwu-wymiarowych obrazów $I$ można zdefiniować 2-wymiarowe kernele $K$ tak, że __dyskretna__ konwolucja będzie zdefiniowana jako
$$S(i,j)=(I*K)(i,j)=\sum_m\sum_nI(m,n)K(i-m,j-n)$$
2. konwolucje są komutatywne, skąd
$$S(i,j)=(K*I)(i,j)=\sum_m\sum_nI(i-m,j-n)K(m,n)$$
  * ten opis jest dla obrazów bardziej oczywisty 
  * zwykle implementowany implementowany będzie schemat korelacji
  $$S(i,j)=(I*K)(i,j)=\sum_m\sum_nI(i+m,j+n)K(m,n)$$
  dający ten sam wynik
  * macierz kernela jest _rzadka_: zerowa wszędzie poza polami leżącymi na obrazie
3. co dają konwolucje?
  * __rzadkie interakcje__
    * w sieciach warstwowych każdy element wyjściowy jest połączony z każdym wejściowym przez jakiś parametr
    * kernel jest __mniejszy__ niż wejście
    * zwiększa wydajność
    * te same cechy są wykrywane w różnych miejscach
    * w głębokich sieciach neurony w głębszych warstwach współpracują z większymi obszarami wejścia
  * __współdzielenie parametrów__
    * w modelach warstwowych każda waga jest użyta __dokładnie raz__ przy liczeniu wyjścia
    * w sieci konwolucyjnej każda waga kernela jest użyta dla __każdego__ elementu wejścia (ew. poza obszarami na brzegu)
      * zamiast uczyć zestawu wag dla każdego wejścia, uczony jest __jeden__ zestaw
  * __równoważne reprezentacje__
    * utworzone reprezentacje $f$ stają się odporne na translacje $t$, bo $f(t(x))=t(f(x))$
    * kernel wykrywający brzegi będzie do zastosowania w różnych miejscach obrazu

## Pooling
<img src="nn_figures/cnn.pdf" width="100%"> [Goodfellow et al.]
1. typowa warstwa sieci CNN składa się z
  * szeregu __konwolucji__ tworzących liniowe aktywacje
  * __nieliniowych aktywacji__ dla wykrywania (detekcji) cech (features)
  * modyfikacji przez __pooling__
2. pooling __zastępuje__ warstość w danym miejscu pewną statystyką dla obszaru wokół
  * __max pooling__ zwraca maksymalną wartość dla pewnego prostokątnego obszaru
    * każde pole tego obszaru może odpowiadać nieco różnej wykrytej cesze
  * __average pooling__ zwraca średnią
  * __L2 norm pooling__ normą kwadratową dla prostokątnego obszaru
  * __weighted pooling__ ważoną średnią odległość od piksela w centrum obszaru
3. max pooling wprowadza pewną inwariantność na translację
  * niewielkie przesunięcie wejścia może zmienić tylko niewielką część wyjścia
  * także pewną inwariantność na powiększenie
  * pooling zmniejsza rozmiar przetwarzanego obszaru
    * w kolejnych warstwach przetwarzane są cechy wyższego poziomu
    * także efektywniejsze obliczeniowo

## Konwolucje w modelach sieci neuronowych
1. konwolucja z __jednym__ kernelem wykrywa tylko __jedną__ cechę (feature), chociaż w wielu miejscach
  * w sieciach neuronowych potrzebujemy wykrywać __wiele różnych__ cech w wielu miejscach
2. wejściowy obraz składa się zwykle z wektorów obserwacji w każdym punkcie
  * np. obraz RGB
  * wejściem do konwolucji w kolejnej warstwie są wyjścia konwolucji poprzedniej 
  * dla obrazów wejścia i wyjścia są tensorami 3-D
    * jeden indeks to kanał
    * dwa indeksy podają współrzędne
  * w rzeczywistości 4-D tensory: jeszcze indeks pozycji w batchu
3. __stride__ określa przesunięcie ponad niektórymi cechami obrazu wejściowego
  * konwolucja ze stride jest równoważna pełnej konwolucji z downsamplingiem bezpośrednio później
4. __padding__ pozwala na kontrolę szerokości obrazu w kolejnych warstwach
  * bez paddingu obraz zmniejsza się co najmniej o piksel na warstwę
  * bez paddingu albo obraz szybko się zmniejsza albo potrzebne jest użycie małych kerneli
    * oba rozwiązania są niedobre
5. możliwe alternatywy
  * __no (valid) padding__ i pixele kernela __nie mogą __ wychodzić poza obszar obrazu
    * wszystkie piksele wyjścia są funkcją __tej samej__ liczby pikseli wejścia
    * każda kolejna warstwa zmniejsza się
    
    <img width="150px" src="nn_figures/gif/no_padding_no_strides.gif"><img width="150px" src="nn_figures/gif/no_padding_strides.gif">[Dumoulin, Visin, arXiv:1603.07285]
    * liczba pikseli wyjściowych wynosi (dla $s=1$) $$o=(i-k)+2p+1$$
    * dla $s>1$ liczba pikseli wyjściowych $$o=\left\lfloor\frac{i-k}{s}\right\rfloor+1$$
  * __same (half) padding__ zapewnia tyle dodanych pikseli by warstwy __nie zmniejszały__ się
    * może być dowolna liczba warstw konwolucji
    * piksele blisko brzegu wpływają na mniej pikseli wyjściowych
    
    <img width="150px" src="nn_figures/gif/same_padding_no_strides.gif">[Dumoulin, Visin, arXiv:1603.07285]
    * liczba pikseli wyjściowych (dla $s=1, k=2n+1, p=\lfloor{}k/2\rfloor$) to $$o=i+2\lfloor{}k/2\rfloor-(k-1)=i+2n-2n=i$$
  * __full padding__ gdzie dodane jest tyle zer, by każdy piksel obrazu był odwiedzony tą samą liczbę razy
    * każdy __wyjściowy__ piksel blisko brzegu jest funkcją mniejszej liczby pikseli wejściowych
    
    <img width="150px" src="nn_figures/gif/full_padding_no_strides.gif">[Dumoulin, Visin, arXiv:1603.07285]
    * liczba pikseli wyjściowych (dla $p=k-1, s=1$) to $$o=i+2(k-1)-(k-1)=i+(k-1)$$
    
    <img width="150px" src="nn_figures/gif/padding_strides.gif"><img width="150px" src="nn_figures/gif/padding_strides_odd.gif">[Dumoulin, Visin, arXiv:1603.07285]
  * w ogólnym przypadku liczba wyjsciowych to $$o=\left\lfloor\frac{i+2p-k}{s}\right\rfloor+1$$
6. zwykle optymalne rozwiązanie leży gdzieś między _valid_ a _same_

## Pooling
1. warstwy pooling zapewniają pewne niewielkie inwariancje na translacje wejścia
2. __max pooling__ 
  * dzieli wejście na patche
    * zwykle __nie__ nakładające się
  * wybraniu maksymalnej wartości
3. pooling nie wykorzystuje paddowania, a więc $$o=\left\lfloor\frac{i-k}{s}\right\rfloor+1$$
  * a więc tak samo jak konwolucje bez paddingu

## Dekonwolucje
1. a gdybyśmy chcieli odzyskać widzialne obrazy z głębokich reprezentacji?
  * prostym rozwiązaniem jest mnożenie przez __transpozycję__ macierzy konwolucji
  * modele auto-enkoderów, RBM, itp.
2. jest to jednak trochę bardziej złożone niż w modelach warstwowych
3. konwolucje __transponowane__

  <img width="150px" src="nn_figures/gif/no_padding_no_strides.gif"><img width="150px" src="nn_figures/gif/no_padding_no_strides_transposed.gif">[Dumoulin, Visin, arXiv:1603.07285]
  * konwolucja $3\times3$ nad wejsciem $4\times4$ bez paddingu ($p=0$) i jednostkowym stridem ($s=1$)
  * równoważne konwolucji $3\times3$ nad wejściem $2\times2$ wypadowane z każdej strony pasem $2$ pikseli ($p=2$) z jednostkowym strajdem (???!!!???) ($s=1$)
  * jednak narożne piksele wejścia wpływają __jedynie__ na naróżne piksele odtwarzanego obrazu
    * paddowanie ma wymiar $p=k-1$
    * stąd wymiar wyjściowy to $o'=i'+(k-1)$
  * takie transponowane konwolucje zresztą jedynie odtwarzają __kształt__, a nie ma tu żadnej pewności odtwarzania wejścia
4. konwolucje strajdowane __cząstkowo__ (_ang_. __fractionally__)
  * czy możemy sobie wyobrazić sytuację z $s<1$??
  
  <img width="150px" src="nn_figures/gif/no_padding_strides_transposed.gif"><img width="150px" src="nn_figures/gif/padding_strides_transposed.gif">[Dumoulin, Visin, arXiv:1603.07285]
  * wyjście będzie miało wymiar $$o'=s(i'-1)+k$$ gdzie $p'=k-1$ a rozszerzone wejście $i'$ jest uzyskane przez dodanie $s-1$ pasów zer miedzy wszystkimi zerami/kolumnami

## Konwolucje rozmyte (dilated)
1. albo __atrous__ z francuskiego ___a trous___ (_z dziurami_)
2. rozmyciu ulega kernel przez dodanie spacji pomiędzy elementy jądra
  * dodawane jest $d-1$ rzędów/wierszy spacji, gdzie $d=1$ odpowiada konwolucji nierozmytej
  * rozmycie kernela sztucznie zwiększa jego wymiar do $$k'=k+(k-1)(d-1)$$
  <img width="150px" src="nn_figures/gif/dilation.gif">[Dumoulin, Visin, arXiv:1603.07285]

## Uczenie sieci CNN
1. każdy kernel jest odpowiedzialny za wykrywanie pewnej cechy
2. okazuje się, że wsteczna propagacja jest __wystarczająca__
  * konwolucja
  * wsteczna propagacja od wyjścia do wag
  * wsteczna propagacja od wyjscia do wejścia
  
  są wystarczające dla nauczenie dowolnej sieci CNN z propagacją wprzód
  
3. __bias__ jest także elementem sieci CNN
  * w sieciach warstwowych typowy bias jest związany z każdym neuronem
  * w CNN każdy bias jest typowo związany z każdym kanałem
    * możliwe jest rozróżnienie biasu dla różnych położeń obrazu

# ImageNet: Alex net
<img src="nn_figures/imagenet.pdf" width="100%">[Krizhevsky, Sutskever, Hinton NIPS]
1. Konkurs ILSVRC'2010 (ImageNet Large-Scale Visual Recognition Challenge)
  * oryginalny zbiór danych ma ponad 22 miliony obrazów etykietowanych ponad 22 tysiącami klas
  * zbiór danych konkursu to 1.2 miliona obrazów z 1000 różnych klas
  * 50 tysięcy walidacyjnych, 150 tysięcy testowych
  * obrazy różnych rozdzielczości przeskalowane i wycięte do $256\times256$
    * w końcowym uczeniu użyte obrazy $224\times224$
  * dwie miary błędów
    * __top-1__ binarna miara prawidłowa/nieprawidłowa
    * __top-5__ że prawidłowa etykieta __jest/nie jest__ wśród 5-ciu zaproponowanych
  * problem uczony na danych __jedynie__ z odjętą średnią dla każdego piksela/kanału
2. wyniki
  * top-1 37.5\% błędów
  * top-5 17.0\% błędów
    * jedna z wersji osiągnęła 15.3\% w top-5
    * drugi najlepszy miał w top-5 26.2\% błędów

## Architektura
<img src="nn_figures/imagenet.pdf" width="100%">[Krizhevsky, Sutskever, Hinton NIPS]
1. 8 warstw: 5 konwolucyjnych i 3 warstwowe
1. ostatnia warstwa jest softmaxem o wymiarze 1000 zwracającym szanse dla kazdej z klas
2. kolejne warstwy konwolucyjne
  * pierwsza $224\times224\times3$ ma 96 kerneli o wymiarach $11\times11\times3$ i stride $s=4$
  * druga na obrazie $55\times55\times48$ ma 256 kerneli $5\times5\times48$
    * __uwaga__ warstwy konwolucyjne zostały zaimplementowane na dwóch GPU (Nvidia GTX-580 z 3GB pamięci każda)
    * skutkuje to __podziałem__ kanałów na dwa procesory
    * kernele w warstwie 3 biorą wejście z __wszystkich__ kerneli warstwy 2
    * jednak kernele warstwy 4 biorą wejście __jedynie__ z kerneli na __tym samym__ GPU
    * to powoduje zmianę głębokości
    * ten schemat ulepszył wyniki po ok. 1.5\% dla uzytych błędów
  * trzecia warstwa konwolucji na __dwóch__ obrazach $27\times27\times128$ używa 384 kerneli $3\times3\times256$
  * czwarta ma 384 kernele $3\times3\times192$ na obrazie $13\times13\times192$
  * piąta ma 256 kerneli $3\times3\times192$ na obrazie $13\times13\times192$ dając dwa obrazy $13\times13\times128$
3. warstwy max-pooling użyte są po pierwszej, po drugiej i po piątej konwolucji
  * wbrew zwyczajowi warstwy te są __nakładające__ się, co minimalnie zmniejsza wymiar
4. po piatej warstwie następują dwie warstwy po 4096 neuronów każda
  * warstwy też podzielone pomiedzy dwa GPU
5. łącznie sieć ma ponad 60 milionów parametrów
  * 

## Overfitting
1. w trakcie uczenia sieć overfitowała
  * mimo ponad miliona przykładów
2. augmentacja danych
  * z oryginalnych $256\times256$ wybrane losowo obrazy $224\times224$
  * także losowe poziome odbicia
  * obrazy generowane na bierząco
  * 2048 razy więcej obrazów
3. w trakcie testowania
  * wybrane obrazy z rogów
  * obraz centralny
  * odbicia poziome każdego z nich
  * uśrednione $10$ znalezionych predykcji
4. modyfikacja intensywności kanałów RGB
  * PCA na kanałach
  * do każdego kanału dodana wielokrotność znalezionej składowej głównej przemnożonej przez warstość własną przez wartość losową z rokładu normalnego
    * do każdego pksela $I_{xy}=[I_{xy}^R,I_{xy}^G,I_{xy}^B]$ dodana wartość $$[p_1,p_2,p_3][\alpha_1\lambda_1,\alpha_2\lambda_2,\alpha_3\lambda_3]^T$$ gdzie $p_i,\lambda_i$ to $i$-te wektory i składowe główne macierzy kowariancji $x\times3$ wartosci RGB
  * ten schemat minimalizuje wpływ oświetlenia
  * polepszył on wyniki top-1 o ponad 1\%

## Dropout
1. użyty w pierwszych warstwach sieci z pełnymi połączeniami
2. użyte $p=0.5$
3. wymagało dwukrotnie dłuższego czasu nauczania

## Nauczone filtry
<img src="nn_figures/imagenet-filters.pdf" width="100%">[Krizhevsky, Sutskever, Hinton NIPS]
1. model nauczył się szeregu różnych filtrów
2. w warstwie pierwszej
  * GPU1 nauczył się filtrów praktycznie nie zawierających kolorów, za to posiadających cechy wykrywające krawędzie i zwroty
  * GPU2 wykrywał bloby kolorów
  * według autorów następowało to zawsze __bez względu__ na inicjalizację


## Obrazy
<img src="nn_figures/imagenet-images.pdf" width="100%">[Krizhevsky, Sutskever, Hinton NIPS]

1. po lewej obrazy i ich klasyfikacje przez model
2. po prawej obrazy testowe w pierwszej kolumnie i po pięć obrazów najbliższych w euklidesowej przestrzeni ostatniej warstwy ukrytej
3. usunięcie którejkolwiek warstwy środkowej powodowało spadek skuteczności o ponad 2%
  * głębokość gra rolę

CIFAR-10 recognition [cs232n.stanford.edu](http://cs231n.stanford.edu).

# VGG-16 (Simonyan, Zisserman, ImageNet'2014)
<img src="nn_figures/vgg16.png" width="100%"> [Simonyan, Zisserman, 2014]
1. Krizhevsky et al. zwracali uwagę, że głębokość gra rolę
2. jednocześnie wydawało się, że konwolucje muszą być dużę na początku, by obejmować większy semantycznie obszar
## Architektura
3. jeśli jednak weźmiemy __mały__ wymiar kernela, to przy małym paddingu można utrzymać wymiar
  * $k=3$, $p=\lfloor3/2\rfloor$ i wtedy $$o=i+2\lfloor k/2\rfloor-(k-1)=i+2\left\lfloor\frac{2n+1}{2}\right\rfloor-(2n+1-1)=i+n-n=i$$
  * to pozwala na dużą liczbę warstw
4. kernel $3\times3$ jest najmniejszym pozwalającym na wykrywanie krawędzi i pojęć lewy/prawy, góra/dół
5. wejście stałe $224\times224$ z odjętymi średnimi pikseli (jak w AlexNet)
6. architektura na rysunku ma 16 warstw konwolucyjnych (wszystkie $3\times3$) w blokach
  1. $3\times3\times64$, ReLU, $3\times3\times64$, ReLU, max-pool
  2. $3\times3\times128$, ReLU, $3\times3\times128$, ReLU, max-pool
  3. $3\times3\times256$, ReLU, $3\times3\times256$, ReLU, $3\times3\times256$, ReLU,  max-pool
  4. $3\times3\times256$, ReLU, $3\times3\times512$, ReLU, $3\times3\times512$, ReLU,  max-pool
  5. $3\times3\times256$, ReLU, $3\times3\times512$, ReLU, $3\times3\times512$, ReLU,  max-pool
  6. FF 4096, FF 4096, FF 1000, softmax
7. rozszerzona sieć z 19 warstwami
  1. $3\times3\times64$, ReLU, $3\times3\times64$, ReLU, max-pool
  2. $3\times3\times128$, ReLU, $3\times3\times128$, ReLU, max-pool
  3. $3\times3\times256$, ReLU, $3\times3\times256$, ReLU, $3\times3\times256$, ReLU,  max-pool
  4. $3\times3\times256$, ReLU, $3\times3\times512$, ReLU, $3\times3\times512$, ReLU, $3\times3\times512$, ReLU,  max-pool
  5. $3\times3\times256$, ReLU, $3\times3\times512$, ReLU, $3\times3\times512$, ReLU, $3\times3\times512$, ReLU, max-pool
  6. FF 4096, FF 4096, FF 1000, softmax
7. w jeszcze innej konfiguracji z 16 warstwami ostatnie konwolucje w dwóch ostatnich blokach (4. i 5.) były o wymiarach $1\times1$
  * konwolucje $1\times1$ w zasadzie nie wprowadzają żadnej informacji o sąsiedztwie
  * jednak zwiększają nieliniowość __nie__ zmieniając wymiaru
## Uczenie
1. sieć ma większą liczbę parametrów niż AlexNet 
  * ok. 140 milionów
2. uczenie SGD z momentum rozpoczęło się ze współczynnikiem uczenia $0.01$ by być zmniejszonym gdy spadek na zbiorze walidacyjnym zanikł
3. uczenie zostało zatrzymane po 370 tysiącach iteracji
4. model był uczony na dwóch rozdzielczościach: 256 i 384 pikseli
## Wyniki
1. wersje 16- i 19-warstwowe osiągały do 24.8% w top-1 i 7.5% w top-5
2. wersja złożona z 2 modeli osiągała 23.7% oraz 6.8% (7.32% w złożonym modelu)
3. to wyniki porównywalne ze zwycięzcą GoogleNet, który miał 6.67%
4. a model vgg-16 jest znacznie prostszy...

# GoogleNet (Szegedy et al.)
<img src="nn_figures/inception-naive.pdf" width="50%"><img src="nn_figures/inception-reduction.pdf" width="50%">[Szegedy et al. 2015]
1. ten model rozwinął głębokość
2. cały model zbudowany jest z modułów __inception__
  * wszystkie konwolucje mają wymiary $5\times5$, $3\times3$ i $1\times1$
  * wymiary podyktowane wygodą, by łatwiej się zgadzały granice
  * każdy moduł jest połączeniem wielu konwolucji wraz z poolingiem
    * ekstrakcja cech na wielu poziomach jednocześnie
    * $1\times1$ pozwala na konwolucje nie przestrzenną (spatial), ale tzw. __cross-channel__
3. dla zapewnienia inwariantności na translacje uzywamy powtarzalnych bloków
  * blok ma analizować korelacje w warstwie i grupować je z dużą korelacją
  * te tworzą elementy kolejnej warstwy
  * klastry skoncentrowane w jednym regionie mogą być łatwo przetwarzane przez konwolucje $1\times1$ dla zwiększenia nieliniowości
  * bardziej rozległe cechy muszą być pokryte przez kernele o większym wymiarze
  * dodatkowy pooling też daje zyski
  * tak wygląda _naiwny_ moduł inception
4. poważnym problemem tak poustawianych jeden na drugim modułów będzie coraz większe rozrzucenie przestrzenne cech wyższego rzędu
  * to sugeruje zwiększenie frakcji udziału konwolucji $3\times3$ i $5\times5$ na wyższych poziomach
  * a to staje się coraz bardziej kosztowne na wyższych poziomach
  * łączenie wyjścia poolingu oraz konwolucji spowoduje zwiększenie liczby wyjść z każdym krokiem, co będzie nieefektywne
5. rozwiązaniem może być __redukcja__ wymiarowości gdziekolwiek to możliwe
  * wykorzystywane jest tu podejście embeddingu
  * konwolucje $1\times1$ są obliczane przed kosztownymi $3\times3$ i $5\times5$
  * moduł Inception na rysunku po prawej
6. moduły inception są ustawiane jeden po drugim
  * to pozwala na zwiększanie ich liczby __bez__ niekontrolowanego wzrostu złożoności

## GoogLeNet

<img src="nn_figures/googlenet.pdf" width="100%">
1. sieć jest bardzo głęboka
  * propagacja gradientów wgłąb może być problemem
  * aby cały model dawał dobre wyniki, cechy generowane na niższych i środkowych warstwach powinny być dyskryminatywne
2. GoogLeNet dodaje klasyfikatory na niższych i środkowych poziomach
  * powinny wzmacniać poprawne klasyfikacje na tych poziomach
  * zwiększać sygnał gradientu
  * dodawać regularyzację
3. te klasyfikatory
  * są dodane w środkowych modułach 4a i 4d
  * mają postać małych sieci konwolucyjnych
  * ich koszt jest dodawany do całkowitego zdyskontowane (zwykle ich koszty przemnożone przez 0.3)
  * usunięte w trakcie inferencji

## Liczba parametrów
1. AlexNet miał ok. 60 milionów
2. VGG19 osiągnął w ILSVR'2014 wynik minimalnie słabszy od GoogLeNet
3. VGG19 ma zaletę bardzo __prostej__ architektury ale kosztem olbrzymiej liczby parametrów - ok. 180 milionów
4. GoogLeNet ma tylko 5 milionów parametrów
  * powinna być lepsza generalizacja
  * mniej dodatkowych chwytów (augmentacja, itp)

# Dekonwolucje
1. dekonwolucje są w rzeczywistości transponowanymi konwolucjami
2. pozwalają odwrócić cykl
  * cześć konwolucyjna od obrazu do reprezentacji
  * część odwracająca reprezentację do postaci obrazu
  
<img src="nn_figures/101-inp.jpg" width="45%"><img src="nn_figures/101-out.jpg" width="45%">
<img src="nn_figures/699-inp.jpg" width="45%"><img src="nn_figures/699-out.jpg" width="45%"> [za P. Garg]
  * konwolucyjna
    * Conv2d(16, 4) -> ReLU -> MaxPool(2) -> Conv2d(32, 5) -> ReLU -> MaxPool(2) -> Conv2d(64, 3) -> ReLU
  * dekonwolucyjna 
    * ConvTransp2d(32, 3) -> ReLU -> MaxUnPool(2) -> ConvTransp2d(16, 5) -> ReLU -> MaxUnPool(2) -> ConvTransp2d(3, 4) -> ReLU
3. pozwala na odszumianie oraz semantyczną segmentację
4. max-pool __nie jest__ operacją odwracalną
  * jak zrobić un-pooling?
    * zapamiętując indeksy
      * każdy max-pool zapamietuje __która__ z pozycji w kernelu (np. 2x2) miała najwyższą wartość
      * przy odwracaniu ta pozycja dostaje wartość wyjściową, a pozostałe 0
    * a jeśli indeksy __nie są__ dostępne?
      * niech architektura będzie architekturą autoenkodera
      * od wejścia do warstwy latent $Z$ jest sieć konwolucyjna
      * od latent do wyjściowej dekonwolucyjna
      * zadaniem jest __wylosować__ $z\in{}Z$ co nie podaje indeksów
      * najprostszym rozwiązaniem jest __upsampling__

# Network in Network
1. filtr konwolucji jest w rzeczywistości uogólnionym modelem liniowym (GLM)
  * poziom abstrakcji nie jest wysoki
  * konieczne jest tworzenie wielu filtrów
  * problem gdy dane dla pewnego wzorca (concept) leżą na bardziej skomlikowanej powierzchni niż półpłaszczyna
2. warstwa __mlpconv__ wykorzystuje nieliniowy detektor
<img src="nn_figures/nin.pdf" width="100%">
  * filtr konwolucyjny wraz z nieliniową aktywacją, np. ReLU, oblicza $$f_{ijk}\max(w_K^Tx_{ij},0)$$
    * naturalnie obliczana jest liczba warstw zgodna z definicją MLP - może być głęboką siecią
  * podobnie mlpconv mapuje pewien obszar (patch) do wartości wyjsciowej w następnej warstwie
  * dzieje się tak przez wielowarstwowy perceptron z nielioniowymi aktywacjami
  * MLP jest __wspólny__ dla wszystkich lokalnych pól wejściowych
  * średnia ostatniej warstwy jest przekazywana jako wartość zaufania do obliczonych cech
  * to przypomina $maxout$ gdzie wybierane jest maksimum z wielu obliczonych wartości

## Konwolucje $1\times1$
1. nikt nam nie zabrania ustalenia wymiaru kernela na $k=1$, ale po co...?
2. konwolucja $1\times1$ oblicza __nieliniową__ funkcję na pojedynczej kolumnie pikseli obrazu
  * to może być moduł typu NiN
3. pozwala na efektywną redukcję wymiarowości, tzn. liczby kanałów
  1. 256 kanałów -- konwolucje 1x1 --> 64 kanały -- konwolucje 4x4 --> 256 kanałów
  2. 256 kanałów -- konwolucje 4x4 --> 256 kanałów
  * które rozwiązanie jest szybsze???
4. konwolucje przestrzenne na kanałach przetworzonych wcześniej przez $1\times1$ odpowiadają konwolucją na przestrzeniach osadzonych (__embedding__)
  * takie konwolucje są bardzo wydajne nie tracąc informacji
  * przetwarzania na embeddingach są bardzo popularne np. w NLP

# Densenet
<img src="nn_figures/denseblock.pdf" width="80%"> [Huang, Liu, Weinberger, Maaten]
1. wyjście z każdej warstwy (lub ich grupy) jest podłączone do __wszystkich późniejszych__ warstw
  * połączenie jako 
    * konkatenacja wszystkich poprzednich warstw
    * batch normalization
    * ReLU
    * konwolucja $3\times3$
2. ponieważ konkatenacja nie byłaby możliwa przy zmianie rozmiarów map warstwy są poddawana poolingowi w warstwach dostosowujących
  * batch normalization
  * konwolucje $1\times1$
  * $2\times2$ average pooling
3. przed konwolucjami $3\times3$ dodawane są konwolucje $1\times1$ dla zmniejszenia wymiarowości i poprawie efektywności
<img src="nn_figures/densenet.png" width="100%"> [Huang, Liu, Weinberger, Maaten]
4. DenseNet z różnymi parametrami (wielkościami warstw, ich głębokościami, etc) osiągał (wg. metodologii takiej jak w AlexNet) na zbiorze ILSVRC'2012
  * top-1: 23.6%, 22.1%, 21.5%, 20.8%
  * top-5: 6.66%, 5.92%, 5.44%, 5.29%

# Squeezenet
<img src="nn_figures/squeezenet.png" width="100%"> [Indola, Han, Moskewicz, Ashraf, Dally, Keutzer]
1. celem autorów było __zmniejszenie__ sieci CNN
  * łatwiej jest uczyć małe modele w równoległym środowisku
  * prostsza aktualizacja nauczonych modeli, np. automatyczna przez Internet
  * możliwość implementacji na FPGA
2. próba rozwiązania w Squeezenet wykorzystując
  * zamiana dużej liczby konwolucji $3\times3$ na konwolucje $1\times1$
  * zmniejszenie liczby kanałów wejściowych dla konwolucji $3\times3$
    * przez wykorzystanie warstw __squeeze__
  * downsampling (pooling) __późno__ w architekturze, by konwolucje miały większe mapy aktywacji, co prowadzi do niższych błędów
3. moduł __fire__
<img src="nn_figures/squeezenet-fire.pdf" width="60%"> [Indola, Han, Moskewicz, Ashraf, Dally, Keutzer]
  * moduł zgniatający z konwolucji $1\times1$
  * przekazujący wyjście do konwolucji $1\times1$ i $3\times3$
  * pozwala na zmniejszenie liczby kanałów wejsciowych do konwolucji $3\times3$
4. końcowa warstwa __nie ma__ pełnych połączeń, a tylko _average pooling_ jak w NiN
5. metoda kompresji sieci zastosowana do
  * AlexNet
    * SVD zmniejsza sieć 5-krotnie i top-1 do 56%
    * Network Preuning 9-krotnie, top-1 do 57.2%, top-5 do 29.8%
    * Deep Compression zmniejsza AlexNet 35-krotnie bez zmiany poziomu błędu
  * SqueezeNet osiąga 50-krotną redukcję względem AlexNet __bez zmiany__ poziomów błędu
    * Deep Compression z 6-bitową kwantyzacją i 33% rzadkością daje model o wielkości ok. 0.47MB
      * ponad 500 razy mniejszy niż AlexNet!!!